In [15]:
import pandas as pd
import numpy as np

In [3]:
tract_data = pd.read_csv('./CensusData/sfo data/census_data_plus_affordability.csv')

In [6]:
print tract_data.columns[-20:]

Index([u'num_lep_1990', u'USPS', u'POP10', u'HU10', u'ALAND', u'AWATER',
       u'ALAND_SQMI', u'AWATER_SQMI', u'INTPTLAT',
       u'INTPTLONG                                                                                                                  ',
       u'hispanic', u'white', u'black', u'american_indian', u'asian',
       u'pac_islander', u'other_races', u'two_races', u'hh_type1_h',
       u'hh_type1_t'],
      dtype='object')


In [11]:
lats = tract_data.loc[:, ['INTPTLAT', 'INTPTLONG']]

array([329], dtype=int64)

In [18]:
new_columns = tract_data.columns.values
new_columns[np.flatnonzero(tract_data.columns.str.startswith('INTPTLONG'))[0]] = 'INTPTLONG'
tract_data.columns = new_columns
lats = tract_data.loc[:, ['INTPTLAT', 'INTPTLONG']]
tract_data.to_csv('./CensusData/sfo data/new_tract_data.csv')

#### Adding Public Schools Data

In [28]:
from density_estimation import *

In [35]:
pts = lats.loc[20, :].values

In [36]:
pub_schools = pd.read_csv('./Schools/schools_public_pt.csv')

In [40]:
df_schools_loc = pub_schools['the_geom'].str.extract('POINT \((.*) (.*)\)')

c:\users\srinivas\anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [41]:
df_schools_loc = df_schools_loc.iloc[:, ::-1]

In [43]:
df_schools_loc.columns = ['Latitude', 'Longitude']

In [46]:
df_schools_loc = df_schools_loc.astype(float)

In [49]:
scores = np.ones(df_schools_loc.shape[0])

In [56]:
point_density(pts, df_schools_loc.values, kernel='gaussian', bw=1000.0)

1.9492909765348765

In [69]:
scores = np.zeros(lats.shape[0])
for i, p in enumerate(lats.values):
    scores[i] = point_density(p, df_schools_loc.values, kernel='gaussian', bw=100.0)

In [70]:
from bqplot import pyplot as pl

In [71]:
pl.figure()
pl.hist(scores)
pl.show()

In [72]:
tract_data['pub_school_score'] = scores

#### Adding Private Schools

In [73]:
private_schools = pd.read_csv('./Schools/pr_schools.csv')

In [74]:
pr_school_data = private_schools.values

In [79]:
pr_scores = np.zeros(lats.shape[0])
for i, p in enumerate(lats.values):
    pr_scores[i] = point_density(p, pr_scores, kernel='gaussian', bw=100.0)

In [81]:
tract_data['pr_school_score'] = pr_scores

In [82]:
tract_data.to_csv('./CensusData/sfo data/tract_data_with_schools.csv')